In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:95% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

# <span style="color:red"> Step.1_PDF (과실비율인정기준) 데이터 추출 </span>

In [2]:
import pdfplumber
import pandas as pd
import re
import os

def extract_fault_standards_pdf(pdf_path):
    """
    첫 번째 PDF (과실비율인정기준)에서 데이터 추출
    - 사고유형코드, 기본과실비율, 사고유형명, 관련판례 등 추출
    """
    print(f"📖 첫 번째 PDF 처리 시작: {os.path.basename(pdf_path)}")
    
    extracted_data = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            print(f"   📄 페이지 {page_num + 1} 처리 중...")
            
            text = page.extract_text()
            if not text:
                continue
                
            # 페이지별 데이터 추출 시도
            page_data = extract_page_data_standards(text, page_num + 1)
            if page_data:
                extracted_data.append(page_data)
    
    print(f"✅ 첫 번째 PDF 완료: 총 {len(extracted_data)}개 사례 추출")
    return extracted_data

def extract_page_data_standards(text, page_num):
    """
    과실비율인정기준 PDF의 각 페이지에서 데이터 추출
    """
    result = {
        'source_pdf': '과실비율인정기준',
        'page_number': page_num,
        'accident_code': '',
        'accident_name': '',
        'basic_fault_ratio': '',
        'modification_factors': '',
        'related_cases': '',
        'raw_text': text[:500]  # 디버깅용 원본 텍스트 일부
    }
    
    lines = text.split('\n')
    
    # 1. 사고유형코드 찾기 (예: "보1", "차1" 등)
    for line in lines:
        # "보1", "보2", "차1", "차2" 패턴 찾기
        code_match = re.search(r'([보차]\d+)', line)
        if code_match:
            result['accident_code'] = code_match.group(1)
            # 사고유형명도 같은 줄에서 찾기
            # 예: "보1  보행자 직선신호 횡단게시, 직선신호 충격 사고"
            parts = line.split()
            if len(parts) > 1:
                result['accident_name'] = ' '.join(parts[1:])
            break
    
    # 2. 기본과실비율 찾기 (숫자 패턴)
    for line in lines:
        # "70", "기본과실비율 70" 같은 패턴
        ratio_match = re.search(r'기본과실비율.*?(\d+)', line)
        if ratio_match:
            result['basic_fault_ratio'] = ratio_match.group(1)
            break
        # 단순히 숫자만 있는 경우도 체크
        elif re.search(r'^\s*\d{1,2}\s*$', line):
            result['basic_fault_ratio'] = line.strip()
    
    # 3. 수정요소 찾기 (①, ②, ③ 패턴)
    modification_list = []
    for line in lines:
        mod_match = re.search(r'[①②③④⑤]\s*(.+?)\s*([+\-]\d+)', line)
        if mod_match:
            factor = mod_match.group(1).strip()
            value = mod_match.group(2)
            modification_list.append(f"{factor}:{value}")
    
    if modification_list:
        result['modification_factors'] = '; '.join(modification_list)
    
    # 4. 관련판례 찾기 (대법원 YYYY.MM.DD 패턴)
    case_pattern = r'대법원\s*(\d{4}\.\d{1,2}\.\d{1,2}\.?\s*선고\s*\d+[가-힣]\d+\s*판결)'
    case_matches = re.findall(case_pattern, text)
    if case_matches:
        result['related_cases'] = '; '.join(case_matches)
    
    # 데이터가 있으면 반환, 없으면 None
    if result['accident_code'] or result['basic_fault_ratio']:
        return result
    return None

# 실행 예시
if __name__ == "__main__":
    pdf_path_1 = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.pdf"
    
    # Step 1 실행
    data_standards = extract_fault_standards_pdf(pdf_path_1)
    
    # 결과 확인
    if data_standards:
        df_temp = pd.DataFrame(data_standards)
        print("\n📊 추출된 데이터 미리보기:")
        print(df_temp[['accident_code', 'accident_name', 'basic_fault_ratio']].head())
        
        # 임시 저장
        df_temp.to_csv("temp_step1_standards.csv", index=False, encoding='utf-8-sig')
        print("💾 Step1 결과가 temp_step1_standards.csv로 저장되었습니다.")
    else:
        print("❌ 추출된 데이터가 없습니다. 패턴을 확인해주세요.")

📖 첫 번째 PDF 처리 시작: 231107_과실비율인정기준_온라인용.pdf
   📄 페이지 1 처리 중...
   📄 페이지 2 처리 중...
   📄 페이지 3 처리 중...
   📄 페이지 4 처리 중...
   📄 페이지 5 처리 중...
   📄 페이지 6 처리 중...
   📄 페이지 7 처리 중...
   📄 페이지 8 처리 중...
   📄 페이지 9 처리 중...
   📄 페이지 10 처리 중...
   📄 페이지 11 처리 중...
   📄 페이지 12 처리 중...
   📄 페이지 13 처리 중...
   📄 페이지 14 처리 중...
   📄 페이지 15 처리 중...
   📄 페이지 16 처리 중...
   📄 페이지 17 처리 중...
   📄 페이지 18 처리 중...
   📄 페이지 19 처리 중...
   📄 페이지 20 처리 중...
   📄 페이지 21 처리 중...
   📄 페이지 22 처리 중...
   📄 페이지 23 처리 중...
   📄 페이지 24 처리 중...
   📄 페이지 25 처리 중...
   📄 페이지 26 처리 중...
   📄 페이지 27 처리 중...
   📄 페이지 28 처리 중...
   📄 페이지 29 처리 중...
   📄 페이지 30 처리 중...
   📄 페이지 31 처리 중...
   📄 페이지 32 처리 중...
   📄 페이지 33 처리 중...
   📄 페이지 34 처리 중...
   📄 페이지 35 처리 중...
   📄 페이지 36 처리 중...
   📄 페이지 37 처리 중...
   📄 페이지 38 처리 중...
   📄 페이지 39 처리 중...
   📄 페이지 40 처리 중...
   📄 페이지 41 처리 중...
   📄 페이지 42 처리 중...
   📄 페이지 43 처리 중...
   📄 페이지 44 처리 중...
   📄 페이지 45 처리 중...
   📄 페이지 46 처리 중...
   📄 페이지 47 처리 중...
   📄 페이지 48 처리 중...
   📄 페

   📄 페이지 381 처리 중...
   📄 페이지 382 처리 중...
   📄 페이지 383 처리 중...
   📄 페이지 384 처리 중...
   📄 페이지 385 처리 중...
   📄 페이지 386 처리 중...
   📄 페이지 387 처리 중...
   📄 페이지 388 처리 중...
   📄 페이지 389 처리 중...
   📄 페이지 390 처리 중...
   📄 페이지 391 처리 중...
   📄 페이지 392 처리 중...
   📄 페이지 393 처리 중...
   📄 페이지 394 처리 중...
   📄 페이지 395 처리 중...
   📄 페이지 396 처리 중...
   📄 페이지 397 처리 중...
   📄 페이지 398 처리 중...
   📄 페이지 399 처리 중...
   📄 페이지 400 처리 중...
   📄 페이지 401 처리 중...
   📄 페이지 402 처리 중...
   📄 페이지 403 처리 중...
   📄 페이지 404 처리 중...
   📄 페이지 405 처리 중...
   📄 페이지 406 처리 중...
   📄 페이지 407 처리 중...
   📄 페이지 408 처리 중...
   📄 페이지 409 처리 중...
   📄 페이지 410 처리 중...
   📄 페이지 411 처리 중...
   📄 페이지 412 처리 중...
   📄 페이지 413 처리 중...
   📄 페이지 414 처리 중...
   📄 페이지 415 처리 중...
   📄 페이지 416 처리 중...
   📄 페이지 417 처리 중...
   📄 페이지 418 처리 중...
   📄 페이지 419 처리 중...
   📄 페이지 420 처리 중...
   📄 페이지 421 처리 중...
   📄 페이지 422 처리 중...
   📄 페이지 423 처리 중...
   📄 페이지 424 처리 중...
   📄 페이지 425 처리 중...
   📄 페이지 426 처리 중...
   📄 페이지 427 처리 중...
   📄 페이지 428 

# 수정보완(1)

In [3]:
import pdfplumber
import pandas as pd
import re
import os

def extract_fault_standards_pdf(pdf_path):
    """
    첫 번째 PDF (과실비율인정기준)에서 데이터 추출 - 개선된 버전
    - 실제 사고유형 상세 페이지만 추출 (목차 제외)
    - 기본과실비율, 수정요소, 관련판례 정확히 추출
    """
    print(f"📖 첫 번째 PDF 처리 시작: {os.path.basename(pdf_path)}")
    
    extracted_data = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            print(f"   📄 페이지 {page_num + 1} 처리 중...")
            
            text = page.extract_text()
            if not text:
                continue
            
            # 목차나 설명 페이지 제외 - 실제 사고유형 상세 페이지만 처리
            if is_detail_page(text):
                page_data = extract_page_data_standards_improved(text, page_num + 1)
                if page_data:
                    extracted_data.append(page_data)
    
    print(f"✅ 첫 번째 PDF 완료: 총 {len(extracted_data)}개 사례 추출")
    return extracted_data

def is_detail_page(text):
    """
    실제 사고유형 상세 페이지인지 판단
    - 목차, 설명, 서론 페이지 제외
    - 기본과실비율, 수정요소가 있는 실제 상세 페이지만 선택
    """
    # 제외할 페이지 키워드
    exclude_keywords = [
        '목차', '서론', '적용 범위', '용어 정의',
        '제3편', '제1장', '제2장', '제3장',
        '............', '.........',  # 목차의 점선들
        'Page', '002', '003', '004'  # 페이지 번호만 있는 경우
    ]
    
    # 포함되어야 할 키워드 (실제 상세 페이지)
    include_keywords = [
        '기본과실비율', '수정요소', '과실비율',
        '①', '②', '③', '④', '⑤',  # 수정요소 번호
        '대법원', '판결',  # 관련판례
        '%'  # 비율 표시
    ]
    
    # 제외 키워드가 많으면 목차 페이지
    exclude_count = sum(1 for keyword in exclude_keywords if keyword in text)
    if exclude_count >= 3:
        return False
    
    # 포함 키워드가 있으면 상세 페이지
    include_count = sum(1 for keyword in include_keywords if keyword in text)
    return include_count >= 2

def extract_page_data_standards_improved(text, page_num):
    """
    개선된 과실비율인정기준 PDF 데이터 추출
    """
    result = {
        'source_pdf': '과실비율인정기준',
        'page_number': page_num,
        'accident_code': '',
        'accident_name': '',
        'basic_fault_ratio': '',
        'modification_factors': '',
        'related_cases': '',
        'raw_text': text[:500]  # 디버깅용
    }
    
    lines = text.split('\n')
    clean_lines = [line.strip() for line in lines if line.strip()]
    
    # 1. 사고유형코드 찾기 (개선된 패턴)
    accident_code_found = False
    for i, line in enumerate(clean_lines):
        # [보1], [차1] 같은 패턴으로 둘러싸인 코드 우선 찾기
        bracket_match = re.search(r'\[([보차이]\d+)\]', line)
        if bracket_match:
            result['accident_code'] = bracket_match.group(1)
            accident_code_found = True
            
            # 같은 줄 또는 근처에서 사고유형명 찾기
            name_candidates = [line.replace(bracket_match.group(0), '').strip()]
            
            # 앞뒤 줄도 확인
            if i > 0:
                name_candidates.append(clean_lines[i-1])
            if i < len(clean_lines) - 1:
                name_candidates.append(clean_lines[i+1])
            
            # 가장 적절한 사고유형명 선택
            for candidate in name_candidates:
                if candidate and not re.search(r'[\d\.]{3,}|페이지|Page', candidate):
                    # 불필요한 부분 제거
                    clean_name = re.sub(r'\[.*?\]|\.{3,}|\d{3,}', '', candidate).strip()
                    if len(clean_name) > 5:  # 너무 짧은 건 제외
                        result['accident_name'] = clean_name[:100]  # 길이 제한
                        break
            break
    
    # 단순 패턴도 시도 (대비책)
    if not accident_code_found:
        for line in clean_lines:
            simple_match = re.search(r'^([보차이]\d+)', line)
            if simple_match:
                result['accident_code'] = simple_match.group(1)
                # 같은 줄에서 사고유형명 추출
                remaining = line[simple_match.end():].strip()
                if remaining:
                    result['accident_name'] = remaining[:100]
                break
    
    # 2. 기본과실비율 찾기 (개선된 패턴)
    ratio_patterns = [
        r'기본과실비율.*?(\d+)',  # "기본과실비율 70"
        r'보행자.*?기본.*?과실비율.*?(\d+)',  # "보행자 기본 과실비율 70"
        r'과실비율.*?(\d{1,2})(?:\s*%)?',  # "과실비율 70%" 또는 "과실비율 70"
        r'(\d{1,2})\s*%',  # 단순히 "70%"
        r'기본.*?(\d{1,2})',  # "기본 70"
    ]
    
    text_joined = ' '.join(clean_lines)
    for pattern in ratio_patterns:
        ratio_match = re.search(pattern, text_joined, re.IGNORECASE)
        if ratio_match:
            ratio_value = int(ratio_match.group(1))
            if 0 <= ratio_value <= 100:  # 유효한 비율만
                result['basic_fault_ratio'] = str(ratio_value)
                break
    
    # 3. 수정요소 찾기 (개선된 패턴)
    modification_list = []
    modification_patterns = [
        r'[①②③④⑤⑥⑦⑧⑨⑩]\s*([^①②③④⑤⑥⑦⑧⑨⑩]+?)(?:[+\-]?\d+|비례)',
        r'(\d+)\.\s*([^0-9\n]+?)(?:[+\-]?\d+|비례)',
        r'([가나다라마바사아자차카타파하])\s*([^가나다라마바사아자차카타파하]+?)(?:[+\-]?\d+|비례)',
    ]
    
    for line in clean_lines:
        for pattern in modification_patterns:
            matches = re.finditer(pattern, line)
            for match in matches:
                if len(match.groups()) >= 2:
                    factor_desc = match.group(2).strip()
                    # 점수 추출
                    score_match = re.search(r'([+\-]?\d+)', line[match.end()-10:match.end()+10])
                    score = score_match.group(1) if score_match else ''
                    
                    if factor_desc and len(factor_desc) > 3:
                        modification_list.append(f"{factor_desc}:{score}")
    
    # 중복 제거 및 정리
    seen = set()
    unique_modifications = []
    for mod in modification_list:
        if mod not in seen and len(mod) > 5:
            seen.add(mod)
            unique_modifications.append(mod)
    
    result['modification_factors'] = '; '.join(unique_modifications[:10])  # 최대 10개
    
    # 4. 관련판례 찾기 (개선된 패턴)
    case_patterns = [
        r'대법원\s*(\d{4}\.\s*\d{1,2}\.\s*\d{1,2}\.?\s*선고\s*\d+[가-힣]\d+\s*판결)',
        r'대법원\s*(\d{4}-\d+-\d+)',  # 간단한 형태
        r'판결\s*(\d{4}\.\d{1,2}\.\d{1,2})',
        r'(\d{4}년.*?판결)',
    ]
    
    case_matches = []
    for pattern in case_patterns:
        matches = re.findall(pattern, text_joined)
        case_matches.extend(matches)
    
    if case_matches:
        # 중복 제거하고 최대 3개까지
        unique_cases = list(dict.fromkeys(case_matches))[:3]
        result['related_cases'] = '; '.join(unique_cases)
    
    # 5. 데이터 유효성 검사
    # 최소한 사고코드나 기본과실비율 중 하나는 있어야 유효한 데이터
    if result['accident_code'] or result['basic_fault_ratio']:
        return result
    
    return None

# 실행 및 검증 함수
def validate_extracted_data(data_list):
    """추출된 데이터 품질 검증"""
    if not data_list:
        return
    
    print(f"\n📊 데이터 품질 검증 보고서")
    print("="*50)
    
    total = len(data_list)
    has_code = sum(1 for d in data_list if d['accident_code'])
    has_name = sum(1 for d in data_list if d['accident_name'])
    has_ratio = sum(1 for d in data_list if d['basic_fault_ratio'])
    has_modification = sum(1 for d in data_list if d['modification_factors'])
    has_cases = sum(1 for d in data_list if d['related_cases'])
    
    print(f"📈 전체 데이터: {total}건")
    print(f"   - 사고코드 있음: {has_code}건 ({has_code/total*100:.1f}%)")
    print(f"   - 사고유형명 있음: {has_name}건 ({has_name/total*100:.1f}%)")
    print(f"   - 기본과실비율 있음: {has_ratio}건 ({has_ratio/total*100:.1f}%)")
    print(f"   - 수정요소 있음: {has_modification}건 ({has_modification/total*100:.1f}%)")
    print(f"   - 관련판례 있음: {has_cases}건 ({has_cases/total*100:.1f}%)")
    
    # 상위 5개 샘플 출력
    print(f"\n📋 추출된 데이터 샘플:")
    for i, item in enumerate(data_list[:5]):
        print(f"\n{i+1}. 사고코드: {item['accident_code']}")
        print(f"   사고유형명: {item['accident_name'][:50]}...")
        print(f"   기본과실비율: {item['basic_fault_ratio']}")
        print(f"   수정요소: {item['modification_factors'][:50]}...")
        print(f"   관련판례: {item['related_cases'][:50]}...")

# 실행 예시
if __name__ == "__main__":
    pdf_path_1 = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.pdf"
    
    # Step 1 실행
    data_standards = extract_fault_standards_pdf(pdf_path_1)
    
    # 데이터 품질 검증
    validate_extracted_data(data_standards)
    
    # 결과 저장
    if data_standards:
        df_temp = pd.DataFrame(data_standards)
        df_temp.to_csv("temp_step1_standards_improved.csv", index=False, encoding='utf-8-sig')
        print(f"\n💾 개선된 Step1 결과가 temp_step1_standards_improved.csv로 저장되었습니다.")
    else:
        print("❌ 추출된 데이터가 없습니다. 패턴을 다시 확인해주세요.")

📖 첫 번째 PDF 처리 시작: 231107_과실비율인정기준_온라인용.pdf
   📄 페이지 1 처리 중...
   📄 페이지 2 처리 중...
   📄 페이지 3 처리 중...
   📄 페이지 4 처리 중...
   📄 페이지 5 처리 중...
   📄 페이지 6 처리 중...
   📄 페이지 7 처리 중...
   📄 페이지 8 처리 중...
   📄 페이지 9 처리 중...
   📄 페이지 10 처리 중...
   📄 페이지 11 처리 중...
   📄 페이지 12 처리 중...
   📄 페이지 13 처리 중...
   📄 페이지 14 처리 중...
   📄 페이지 15 처리 중...
   📄 페이지 16 처리 중...
   📄 페이지 17 처리 중...
   📄 페이지 18 처리 중...
   📄 페이지 19 처리 중...
   📄 페이지 20 처리 중...
   📄 페이지 21 처리 중...
   📄 페이지 22 처리 중...
   📄 페이지 23 처리 중...
   📄 페이지 24 처리 중...
   📄 페이지 25 처리 중...
   📄 페이지 26 처리 중...
   📄 페이지 27 처리 중...
   📄 페이지 28 처리 중...
   📄 페이지 29 처리 중...
   📄 페이지 30 처리 중...
   📄 페이지 31 처리 중...
   📄 페이지 32 처리 중...
   📄 페이지 33 처리 중...
   📄 페이지 34 처리 중...
   📄 페이지 35 처리 중...
   📄 페이지 36 처리 중...
   📄 페이지 37 처리 중...
   📄 페이지 38 처리 중...
   📄 페이지 39 처리 중...
   📄 페이지 40 처리 중...
   📄 페이지 41 처리 중...
   📄 페이지 42 처리 중...
   📄 페이지 43 처리 중...
   📄 페이지 44 처리 중...
   📄 페이지 45 처리 중...
   📄 페이지 46 처리 중...
   📄 페이지 47 처리 중...
   📄 페이지 48 처리 중...
   📄 페

   📄 페이지 382 처리 중...
   📄 페이지 383 처리 중...
   📄 페이지 384 처리 중...
   📄 페이지 385 처리 중...
   📄 페이지 386 처리 중...
   📄 페이지 387 처리 중...
   📄 페이지 388 처리 중...
   📄 페이지 389 처리 중...
   📄 페이지 390 처리 중...
   📄 페이지 391 처리 중...
   📄 페이지 392 처리 중...
   📄 페이지 393 처리 중...
   📄 페이지 394 처리 중...
   📄 페이지 395 처리 중...
   📄 페이지 396 처리 중...
   📄 페이지 397 처리 중...
   📄 페이지 398 처리 중...
   📄 페이지 399 처리 중...
   📄 페이지 400 처리 중...
   📄 페이지 401 처리 중...
   📄 페이지 402 처리 중...
   📄 페이지 403 처리 중...
   📄 페이지 404 처리 중...
   📄 페이지 405 처리 중...
   📄 페이지 406 처리 중...
   📄 페이지 407 처리 중...
   📄 페이지 408 처리 중...
   📄 페이지 409 처리 중...
   📄 페이지 410 처리 중...
   📄 페이지 411 처리 중...
   📄 페이지 412 처리 중...
   📄 페이지 413 처리 중...
   📄 페이지 414 처리 중...
   📄 페이지 415 처리 중...
   📄 페이지 416 처리 중...
   📄 페이지 417 처리 중...
   📄 페이지 418 처리 중...
   📄 페이지 419 처리 중...
   📄 페이지 420 처리 중...
   📄 페이지 421 처리 중...
   📄 페이지 422 처리 중...
   📄 페이지 423 처리 중...
   📄 페이지 424 처리 중...
   📄 페이지 425 처리 중...
   📄 페이지 426 처리 중...
   📄 페이지 427 처리 중...
   📄 페이지 428 처리 중...
   📄 페이지 429 

# 수정개선보완(2)

In [5]:
import pdfplumber
import pandas as pd
import re
import os

def extract_fault_standards_pdf(pdf_path):
    """
    첫 번째 PDF (과실비율인정기준)에서 데이터 추출 - 최종 개선 버전
    - 사고유형명, 관련판례 추출 대폭 개선
    - 페이지별 상세 분석으로 정확도 극대화
    """
    print(f"📖 첫 번째 PDF 처리 시작: {os.path.basename(pdf_path)}")
    
    extracted_data = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            print(f"   📄 페이지 {page_num + 1} 처리 중...")
            
            text = page.extract_text()
            if not text:
                continue
            
            # 실제 사고유형 상세 페이지만 처리
            if is_accident_detail_page(text):
                page_data = extract_page_data_final(text, page_num + 1)
                if page_data:
                    extracted_data.append(page_data)
    
    print(f"✅ 첫 번째 PDF 완료: 총 {len(extracted_data)}개 사례 추출")
    return extracted_data

def is_accident_detail_page(text):
    """
    사고유형 상세 페이지 판별 - 더 정교한 필터링
    """
    # 반드시 제외할 페이지들
    exclude_strong = [
        '목차', '서론', '적용 범위', '용어 정의', 'Page',
        '제3편', '제1장', '제2장', '제3장',
        '..........................'  # 목차 점선
    ]
    
    # 제외할 가능성이 높은 페이지
    exclude_weak = [
        '002', '003', '004', '005',  # 페이지 번호만 있는 경우
        '자동차사고 과실비율 인정기준',
        '과실비율 적용기준'
    ]
    
    # 반드시 포함되어야 할 요소들
    must_have = [
        r'[보차이]\d+',  # 사고코드
        r'\d{1,2}(?:\s*%)?',  # 비율
    ]
    
    # 포함되면 좋은 요소들
    good_to_have = [
        '기본과실비율', '기본 과실비율', '수정요소',
        '①', '②', '③', '④', '⑤',
        '대법원', '판결', '사고 상황',
        '+', '-', '%'
    ]
    
    # 강력 제외 조건
    for exclude in exclude_strong:
        if exclude in text and len(text) < 1000:  # 짧은 텍스트에서 발견되면 제외
            return False
    
    # 필수 조건 확인
    must_count = sum(1 for pattern in must_have if re.search(pattern, text))
    if must_count < 1:
        return False
    
    # 선호 조건 확인
    good_count = sum(1 for keyword in good_to_have if keyword in text)
    weak_count = sum(1 for keyword in exclude_weak if keyword in text)
    
    # 점수 기반 판별
    score = good_count * 2 - weak_count
    return score >= 3

def extract_page_data_final(text, page_num):
    """
    최종 개선된 데이터 추출 함수
    """
    result = {
        'source_pdf': '과실비율인정기준',
        'page_number': page_num,
        'accident_code': '',
        'accident_name': '',
        'basic_fault_ratio': '',
        'modification_factors': '',
        'related_cases': '',
        'raw_text': text[:300]  # 디버깅용
    }
    
    lines = text.split('\n')
    clean_lines = [line.strip() for line in lines if line.strip()]
    text_joined = ' '.join(clean_lines)
    
    # === 1. 사고유형코드 추출 (최우선) ===
    accident_code = extract_accident_code(clean_lines, text_joined)
    if accident_code:
        result['accident_code'] = accident_code
    
    # === 2. 사고유형명 추출 (대폭 개선) ===
    accident_name = extract_accident_name(clean_lines, text_joined, accident_code)
    if accident_name:
        result['accident_name'] = accident_name
    
    # === 3. 기본과실비율 추출 ===
    basic_ratio = extract_basic_fault_ratio(clean_lines, text_joined)
    if basic_ratio:
        result['basic_fault_ratio'] = basic_ratio
    
    # === 4. 수정요소 추출 ===
    modifications = extract_modification_factors(clean_lines, text_joined)
    if modifications:
        result['modification_factors'] = modifications
    
    # === 5. 관련판례 추출 (개선) ===
    cases = extract_related_cases(clean_lines, text_joined)
    if cases:
        result['related_cases'] = cases
    
    # 최소 조건: 사고코드 또는 기본과실비율이 있어야 유효
    if result['accident_code'] or result['basic_fault_ratio']:
        return result
    
    return None

def extract_accident_code(lines, text_joined):
    """사고유형코드 추출"""
    # 패턴 1: [보1], [차1] 형태 (최우선)
    bracket_match = re.search(r'\[([보차이]\d+)\]', text_joined)
    if bracket_match:
        return bracket_match.group(1)
    
    # 패턴 2: 보1), 차1) 형태 
    paren_match = re.search(r'([보차이]\d+)\)', text_joined)
    if paren_match:
        return paren_match.group(1)
    
    # 패턴 3: 줄 시작에 보1, 차1 형태
    for line in lines:
        if re.match(r'^([보차이]\d+)', line):
            return re.match(r'^([보차이]\d+)', line).group(1)
    
    # 패턴 4: 단독으로 존재하는 코드
    standalone_match = re.search(r'\b([보차이]\d+)\b', text_joined)
    if standalone_match:
        return standalone_match.group(1)
    
    return None

def extract_accident_name(lines, text_joined, accident_code):
    """사고유형명 추출 - 대폭 개선"""
    if not accident_code:
        return None
    
    candidates = []
    
    # === 방법 1: 코드 근처 텍스트에서 추출 ===
    code_patterns = [
        rf'\[{accident_code}\]([^[\n]+)',  # [보1] 직진 대 직진
        rf'{accident_code}\)([^)\n]+)',    # 보1) 직진 대 직진  
        rf'{accident_code}([^0-9\n]+)',    # 보1 직진 대 직진
    ]
    
    for pattern in code_patterns:
        match = re.search(pattern, text_joined)
        if match:
            name_candidate = match.group(1).strip()
            # 정제
            name_candidate = re.sub(r'[\.\-\s]{3,}.*', '', name_candidate)  # 점선 제거
            name_candidate = re.sub(r'\d{3,}.*', '', name_candidate)       # 페이지번호 제거
            if 10 <= len(name_candidate) <= 100:  # 적절한 길이
                candidates.append(name_candidate)
    
    # === 방법 2: 사고 설명 패턴 찾기 ===
    accident_patterns = [
        r'(.*?(?:직진|좌회전|우회전|후진|주차).*?(?:대|사고).*?)',
        r'(.*?(?:보행자|자동차|이륜차).*?(?:횡단|충돌|접촉).*?)',
        r'(.*?(?:신호|교차로|횡단보도).*?(?:통과|진입).*?)',
        r'(\([^)]+\)\s*\([^)]+\))',  # (신호등 있음) (직진)
    ]
    
    for line in lines:
        for pattern in accident_patterns:
            match = re.search(pattern, line, re.IGNORECASE)
            if match:
                name_candidate = match.group(1).strip()
                # 불필요한 부분 제거
                name_candidate = re.sub(r'^\d+[\.\)]\s*', '', name_candidate)  # 번호 제거
                name_candidate = re.sub(r'\[.*?\]', '', name_candidate)        # 괄호 제거
                name_candidate = re.sub(r'\.{3,}.*', '', name_candidate)       # 점선 제거
                if 8 <= len(name_candidate) <= 80:
                    candidates.append(name_candidate)
    
    # === 방법 3: 구조화된 텍스트에서 추출 ===
    structure_patterns = [
        r'사고\s*상황[:\s]*([^:\n]+)',
        r'사고\s*유형[:\s]*([^:\n]+)', 
        r'적용\s*기준[:\s]*([^:\n]+)',
    ]
    
    for pattern in structure_patterns:
        match = re.search(pattern, text_joined, re.IGNORECASE)
        if match:
            name_candidate = match.group(1).strip()
            if 10 <= len(name_candidate) <= 100:
                candidates.append(name_candidate)
    
    # === 최적 후보 선택 ===
    if candidates:
        # 중복 제거
        unique_candidates = list(dict.fromkeys(candidates))
        
        # 점수 매기기
        scored_candidates = []
        for candidate in unique_candidates:
            score = 0
            # 키워드 포함 점수
            keywords = ['직진', '좌회전', '우회전', '보행자', '자동차', '교차로', '횡단', '신호', '사고']
            score += sum(3 for keyword in keywords if keyword in candidate)
            
            # 길이 점수 (적당한 길이 선호)
            if 15 <= len(candidate) <= 50:
                score += 5
            elif 10 <= len(candidate) <= 60:
                score += 3
            
            # 특수문자 패널티
            if re.search(r'[\.]{3,}|\d{3,}', candidate):
                score -= 5
            
            scored_candidates.append((candidate, score))
        
        # 가장 높은 점수의 후보 선택
        if scored_candidates:
            best_candidate = max(scored_candidates, key=lambda x: x[1])
            if best_candidate[1] > 0:  # 최소 점수 이상
                return best_candidate[0]
    
    return None

def extract_basic_fault_ratio(lines, text_joined):
    """기본과실비율 추출"""
    patterns = [
        r'기본\s*과실\s*비율[:\s]*(\d{1,2})',
        r'기본[:\s]*(\d{1,2})\s*%?',
        r'과실\s*비율[:\s]*(\d{1,2})',
        r'(\d{1,2})\s*%',
        r'비율[:\s]*(\d{1,2})',
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text_joined, re.IGNORECASE)
        if match:
            ratio = int(match.group(1))
            if 0 <= ratio <= 100:
                return str(ratio)
    
    return None

def extract_modification_factors(lines, text_joined):
    """수정요소 추출"""
    factors = []
    
    # 패턴 1: ①, ②, ③ 형태
    circle_patterns = r'[①②③④⑤⑥⑦⑧⑨⑩]\s*([^①②③④⑤⑥⑦⑧⑨⑩\n]+?)(?:([+\-]\d+)|비례)'
    matches = re.finditer(circle_patterns, text_joined)
    for match in matches:
        desc = match.group(1).strip()
        score = match.group(2) if match.group(2) else ''
        if len(desc) > 3:
            factors.append(f"{desc}:{score}")
    
    # 패턴 2: 가. 나. 다. 형태
    korean_patterns = r'[가나다라마바사아자차카타파하]\.\s*([^가나다라마바사아자차카타파하\n]+?)(?:([+\-]\d+)|비례)'
    matches = re.finditer(korean_patterns, text_joined)
    for match in matches:
        desc = match.group(1).strip()
        score = match.group(2) if match.group(2) else ''
        if len(desc) > 3:
            factors.append(f"{desc}:{score}")
    
    # 패턴 3: 1), 2), 3) 형태
    number_patterns = r'(\d+)\)\s*([^\d\n]+?)(?:([+\-]\d+)|비례)'
    matches = re.finditer(number_patterns, text_joined)
    for match in matches:
        desc = match.group(2).strip()
        score = match.group(3) if match.group(3) else ''
        if len(desc) > 3 and not re.search(r'페이지|Page', desc):
            factors.append(f"{desc}:{score}")
    
    # 중복 제거 및 정리
    unique_factors = []
    seen = set()
    for factor in factors:
        if factor not in seen and len(factor) > 5:
            seen.add(factor)
            unique_factors.append(factor)
    
    return '; '.join(unique_factors[:8]) if unique_factors else None

def extract_related_cases(lines, text_joined):
    """관련판례 추출 - 개선"""
    cases = []
    
    # 패턴들 (더 다양하게)
    patterns = [
        r'대법원\s*(\d{4}\.\s*\d{1,2}\.\s*\d{1,2}\.?\s*선고\s*\d+[가-힣]\d+\s*판결)',
        r'대법원\s*(\d{4}-\d+-\d+)',
        r'대법원\s*(\d{4}\.?\d{1,2}\.?\d{1,2})',  
        r'(\d{4}년\s*\d{1,2}월\s*\d{1,2}일\s*판결)',
        r'판결\s*(\d{4}\.\d{1,2}\.\d{1,2})',
        r'선고\s*(\d+[가-힣]\d+)',
    ]
    
    for pattern in patterns:
        matches = re.findall(pattern, text_joined)
        cases.extend(matches)
    
    # 판례번호 패턴도 찾기
    case_num_patterns = [
        r'(\d+[가-힣]\d+)',  # 2019다12345
        r'(\d{4}[가-힣]\d+)', # 2019가단12345  
    ]
    
    for pattern in case_num_patterns:
        matches = re.findall(pattern, text_joined)
        for match in matches:
            if not any(match in case for case in cases):  # 중복 방지
                cases.append(match)
    
    # 정리 및 중복 제거
    unique_cases = list(dict.fromkeys(cases))[:5]  # 최대 5개
    return '; '.join(unique_cases) if unique_cases else None

# 검증 및 실행 함수
def validate_and_save_data(data_list, filename):
    """데이터 검증 및 저장"""
    if not data_list:
        print("❌ 추출된 데이터가 없습니다.")
        return
    
    df = pd.DataFrame(data_list)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    
    # 상세 분석
    total = len(data_list)
    metrics = {
        'accident_code': sum(1 for d in data_list if d['accident_code']),
        'accident_name': sum(1 for d in data_list if d['accident_name']),
        'basic_fault_ratio': sum(1 for d in data_list if d['basic_fault_ratio']),
        'modification_factors': sum(1 for d in data_list if d['modification_factors']),
        'related_cases': sum(1 for d in data_list if d['related_cases']),
    }
    
    print(f"\n📊 최종 데이터 품질 보고서")
    print("="*50)
    print(f"전체 데이터: {total}건")
    for field, count in metrics.items():
        print(f"{field}: {count}건 ({count/total*100:.1f}%)")
    
    # 베스트 샘플 출력
    print(f"\n🏆 추출 품질이 좋은 상위 5개 샘플:")
    
    # 품질 점수 계산
    scored_data = []
    for item in data_list:
        score = 0
        if item['accident_code']: score += 20
        if item['accident_name']: score += 25
        if item['basic_fault_ratio']: score += 20
        if item['modification_factors']: score += 20
        if item['related_cases']: score += 15
        scored_data.append((item, score))
    
    # 상위 5개 출력
    top_samples = sorted(scored_data, key=lambda x: x[1], reverse=True)[:5]
    for i, (item, score) in enumerate(top_samples):
        print(f"\n{i+1}. [품질점수: {score}/100]")
        print(f"   코드: {item['accident_code']}")
        print(f"   사고명: {item['accident_name'][:60]}...")
        print(f"   기본비율: {item['basic_fault_ratio']}")
        print(f"   수정요소: {item['modification_factors'][:50]}...")
        print(f"   관련판례: {item['related_cases'][:50]}...")

# 실행
if __name__ == "__main__":
    pdf_path_1 = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.pdf"
    
    print("🚀 Step 1 최종 개선 버전 실행")
    data_standards = extract_fault_standards_pdf(pdf_path_1)
    
    validate_and_save_data(data_standards, "temp_step1_standards_final.csv")
    print(f"\n💾 최종 결과가 temp_step1_standards_final.csv로 저장되었습니다.")

🚀 Step 1 최종 개선 버전 실행
📖 첫 번째 PDF 처리 시작: 231107_과실비율인정기준_온라인용.pdf
   📄 페이지 1 처리 중...
   📄 페이지 2 처리 중...
   📄 페이지 3 처리 중...
   📄 페이지 4 처리 중...
   📄 페이지 5 처리 중...
   📄 페이지 6 처리 중...
   📄 페이지 7 처리 중...
   📄 페이지 8 처리 중...
   📄 페이지 9 처리 중...
   📄 페이지 10 처리 중...
   📄 페이지 11 처리 중...
   📄 페이지 12 처리 중...
   📄 페이지 13 처리 중...
   📄 페이지 14 처리 중...
   📄 페이지 15 처리 중...
   📄 페이지 16 처리 중...
   📄 페이지 17 처리 중...
   📄 페이지 18 처리 중...
   📄 페이지 19 처리 중...
   📄 페이지 20 처리 중...
   📄 페이지 21 처리 중...
   📄 페이지 22 처리 중...
   📄 페이지 23 처리 중...
   📄 페이지 24 처리 중...
   📄 페이지 25 처리 중...
   📄 페이지 26 처리 중...
   📄 페이지 27 처리 중...
   📄 페이지 28 처리 중...
   📄 페이지 29 처리 중...
   📄 페이지 30 처리 중...
   📄 페이지 31 처리 중...
   📄 페이지 32 처리 중...
   📄 페이지 33 처리 중...
   📄 페이지 34 처리 중...
   📄 페이지 35 처리 중...
   📄 페이지 36 처리 중...
   📄 페이지 37 처리 중...
   📄 페이지 38 처리 중...
   📄 페이지 39 처리 중...
   📄 페이지 40 처리 중...
   📄 페이지 41 처리 중...
   📄 페이지 42 처리 중...
   📄 페이지 43 처리 중...
   📄 페이지 44 처리 중...
   📄 페이지 45 처리 중...
   📄 페이지 46 처리 중...
   📄 페이지 47 처리 중...
   📄 

   📄 페이지 377 처리 중...
   📄 페이지 378 처리 중...
   📄 페이지 379 처리 중...
   📄 페이지 380 처리 중...
   📄 페이지 381 처리 중...
   📄 페이지 382 처리 중...
   📄 페이지 383 처리 중...
   📄 페이지 384 처리 중...
   📄 페이지 385 처리 중...
   📄 페이지 386 처리 중...
   📄 페이지 387 처리 중...
   📄 페이지 388 처리 중...
   📄 페이지 389 처리 중...
   📄 페이지 390 처리 중...
   📄 페이지 391 처리 중...
   📄 페이지 392 처리 중...
   📄 페이지 393 처리 중...
   📄 페이지 394 처리 중...
   📄 페이지 395 처리 중...
   📄 페이지 396 처리 중...
   📄 페이지 397 처리 중...
   📄 페이지 398 처리 중...
   📄 페이지 399 처리 중...
   📄 페이지 400 처리 중...
   📄 페이지 401 처리 중...
   📄 페이지 402 처리 중...
   📄 페이지 403 처리 중...
   📄 페이지 404 처리 중...
   📄 페이지 405 처리 중...
   📄 페이지 406 처리 중...
   📄 페이지 407 처리 중...
   📄 페이지 408 처리 중...
   📄 페이지 409 처리 중...
   📄 페이지 410 처리 중...
   📄 페이지 411 처리 중...
   📄 페이지 412 처리 중...
   📄 페이지 413 처리 중...
   📄 페이지 414 처리 중...
   📄 페이지 415 처리 중...
   📄 페이지 416 처리 중...
   📄 페이지 417 처리 중...
   📄 페이지 418 처리 중...
   📄 페이지 419 처리 중...
   📄 페이지 420 처리 중...
   📄 페이지 421 처리 중...
   📄 페이지 422 처리 중...
   📄 페이지 423 처리 중...
   📄 페이지 424 